In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


import time

# kafka_topic_name1 = "FC_Transaction_Base"
kafka_topic_name = 'FC_Account_Master3'
kafka_bootstrap_servers = 'localhost:9092'

spark = SparkSession \
        .builder \
        .appName("Structured Streaming ") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "earliest") \
        .load()
# fd = spark \
#         .readStream \
#         .format("kafka") \
#         .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
#         .option("subscribe", kafka_topic_name1) \
#         .option("startingOffsets", "earliest") \
#         .load()

df1 = df.selectExpr("CAST(value AS STRING)", "timestamp")
# fd1 = fd.selectExpr("CAST(value AS STRING)", "timestamp")


df_schema_string = "encrypted_account_number STRING, product STRING, product_category STRING"
# fd_schema_string = "order_id INT, account_number STRING, branch STRING, transaction_code STRING"


df2 = df1 \
        .select(from_csv(col("value"), df_schema_string) \
                .alias("data"), "timestamp")
# fd2 = fd1 \
#         .select(from_csv(col("value"), fd_schema_string) \
#                 .alias("data1"), "timestamp")


df3 = df2.select("data.*", "timestamp")

# fd3 = fd2.select("data1.*", "timestamp")
df4 = df3.withColumn('account_number',
                               expr("trim(aes_decrypt(unbase64(encrypted_account_number), '1234567890abcdef', 'ECB', 'PKCS'))")).select('account_number','product','product_category')

    
df4.createOrReplaceTempView("data_find");
song_find_text = spark.sql("SELECT * FROM data_find")
data_agg_write_stream = song_find_text \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("test5Table") \
        .start()

# fd3.createOrReplaceTempView("data1_find");
# song1_find_text = spark.sql("SELECT * FROM data1_find")
# data1_agg_write_stream = song1_find_text \
#         .writeStream \
#         .trigger(processingTime='5 seconds') \
#         .outputMode("append") \
#         .option("truncate", "false") \
#         .format("memory") \
#         .queryName("test2Table") \
#         .start()

data_agg_write_stream.awaitTermination(1)
# data1_agg_write_stream.awaitTermination(1)

24/05/14 11:21:06 WARN Utils: Your hostname, Bisheshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.163.225 instead (on interface en0)
24/05/14 11:21:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/bses/.ivy2/cache
The jars for the packages stored in: /Users/bses/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-14fbaa50-a580-4285-b136-e6df64c0a149;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 234ms :: artifacts dl 7ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#h

False

In [2]:
df = spark.sql("SELECT * FROM test5Table")
df.show(3)

+-------------------+-------+----------------+
|     account_number|product|product_category|
+-------------------+-------+----------------+
|02XYZXYZ10015592101|    SBA|           SBPPS|
|02XYZXYZ10015593701|    SBA|           SBPPS|
|02XYZXYZ10015593801|    SBA|           SBPPS|
+-------------------+-------+----------------+
only showing top 3 rows



In [3]:
df_count = df.count()
df_count

541

In [6]:
fd = spark.sql("SELECT * FROM test2Table")
fd.show(3)

+--------+-------------------+------+----------------+--------------------+
|order_id|     account_number|branch|transaction_code|           timestamp|
+--------+-------------------+------+----------------+--------------------+
|       0|02XYZXYZ10017529992|    15|              CI|2024-05-13 12:44:...|
|       1|02XYZXYZ10017529992|    15|              CI|2024-05-13 12:44:...|
|       2|02XYZXYZ10017517823|    15|              CI|2024-05-13 12:44:...|
+--------+-------------------+------+----------------+--------------------+
only showing top 3 rows



In [7]:
fd_count = fd.count()
fd_count

1163